In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
# Definieer wat waardes voor je k's
k1 = 100000000
k2 = 4000
k3 = 100000000
k4 = 1
k5 = 0
k6 = 0
kcat = 2.1

# Definieer de concentraties op T=0 (in eerste instantie: voor inhibitie of substraat)
E0 = 0.1*10**-9
ES0 = 0
P0 = 0
EI0 = 0
EIi0 = 0  #(EI*)

#Definieer constanten
Is = [100*10**-9, 50*10**-9,# 25*10**-9, 15*10**-9, 7.5*10**-9, 4*10**-9, 2*10**-9, 1*10**-9, 0.5*10**-9, 0.25*10**-9, 
      0.1*10**-9, 0*10**-9] #lijst van concentraties
S0 = 4*10**-6
maxdt = 0.01 #Dit is hoe groot je simulatie tijdstappen maximaal zijn
maxd = 0.01 #Dit is hoeveel je waardes max mogen veranderen per simulatiestap (nu 0.1%)

tframe = 60 #save a frame every 60 s
tincub = 0 #incubate for 1800 s
tsub = 1800 # Run with substrate for 1800 s

In [0]:
all_plots = []


for I in Is: # Loop over all I's
  concentraties = np.array([E0, ES0, P0, EI0, EIi0], dtype=np.double) #matrix van concentraties
  # Voor toevoeging van S

  S=0 # Geen S in je systeem

                          # [         E,       ES, P,     EI, EI*]
  tijdevolutie0 = np.array([[-k3*I-k1*S,  k2+kcat, 0,     k4,   0],  #E
                            [      k1*S, -k2-kcat, 0,      0,   0],  #ES
                            [         0,     kcat, 0,      0,   0],  #P
                            [      k3*I,        0, 0, -k4-k5,  k6],  #EI
                            [         0,        0, 0,     k5, -k6]]) #EI*

  #verandering in E   = [E]*(-k3*I-k1*S) + [ES]*(k2+kcat)  + [P]*(0) + [EI]*(k4)     + [EI*]*(0)
  #verandering in ES  = [E]*(k1*S)       + [ES]*(-k2-kcat) + [P]*(0) + [EI]*(0)      + [EI*]*(0)
  #verandering in P   = [E]*(0)          + [ES]*(kcat)     + [P]*(0) + [EI]*(0)      + [EI*]*(0)
  #verandering in EI  = [E]*(k3*I)       + [ES]*(0)        + [P]*(0) + [EI]*(-k4-k5) + [EI*]*(k6)
  #verandering in EI* = [E]*(0)          + [ES]*(0)        + [P]*(0) + [EI]*(k5)     + [EI*]*(-k6)


  #Na toevoeging van S
  S= S0
  tijdevolutie1 = np.array([[-k3*I-k1*S, k2+kcat, 0, k4, 0],
                           [k1*S, -k2-kcat, 0, 0, 0],
                           [0, kcat, 0, 0, 0],
                           [k3*I, 0, 0, -k4-k5, k6],
                           [0, 0, 0, k5, -k6]])
  # Maak een lijst van kopien van de concentraties
  plot = [concentraties.copy()]

  tijdconc = concentraties.copy()
  frame = 0
  t = 0
  # Voor toevoging van S (tincub/dt stappen = 30min)
  while frame < int(tincub/tframe):
      diff = np.dot(tijdevolutie0, tijdconc)
      dt = np.nanmin([i for i in abs(np.divide(tijdconc, diff)) if i!=0])*maxd #Max 10% change per dt (0s excluded as 0/0=nan, 0/x=0, and abs(x/0)=infinite)
      dt = min(dt, tframe-t, maxdt) # Make sure we don't overshoot the frame
      tijdconc += diff*dt # Doe 1 dt tijdstap
      t += dt
      if t >= tframe:
          plot.append(tijdconc.copy()) # Voeg de nieuwe concentraties toe aan de lijst
          frame += 1
          t=0
  # Na toevoeging van S (tsub/dt stappen)
  t=0
  frame=0
  temp = 0 #temp calculation counter
  old_diff = E0
  old_dt = 1
  while frame < int(tsub/tframe):
      diff = np.dot(tijdevolutie1, tijdconc)
      dt = np.nanmin([i for i in abs(np.divide(tijdconc, diff)) if i!=0])*maxd #Max 10% change per dt (0s excluded as 0/0=nan, 0/x=0, and abs(x/0)=infinite)
      dv = abs(old_diff)*old_dt/abs(diff[0]) #Try to kill fluctuations in E
      if  0 < dv < maxdt and 0 < abs(tijdconc[0]/(diff[0]*maxdt)) < 10**10:
        dv = max(dv, 0.0001)
      else:
        dv = maxdt
      dt = min(dt, tframe-t, maxdt, dv) # Make sure we don't overshoot the frame
      tijdconc += diff*dt # Doe 1 dt tijdstap
      t += dt
      temp += 1
      old_dt = dt
      old_diff = diff[0]
      if t >= tframe:
          plot.append(tijdconc.copy()) # Voeg de nieuwe concentraties toe aan de lijst
          frame += 1
          t=0
  #print(temp)
  print((tsub/0.0001)/temp) #Speedup
  all_plots.append(plot)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in true_divide


6.665903791010584
7.05633582513459


In [0]:
# This will make all the plots and dataframes
dataframes = []
fig, axes = plt.subplots(len(all_plots), 2, figsize=(12,len(all_plots)*6))
for n, plot in enumerate(all_plots):
  # Splits de lijst van figuren in E, ES, P, EI en EI*
  Eplot = [] 
  ESplot = []
  Pplot = []
  EIplot = []
  EIiplot = []
  EItot = []
  for frame in plot:
    Eplot.append(frame[0])
    ESplot.append(frame[1])
    Pplot.append(frame[2])
    EIplot.append(frame[3])
    EIiplot.append(frame[4])
    EItot.append(frame[3]+frame[4]) 
    
  # Plot de figuren
  labels = ('E', 'ES', "P", "EI", "EI*", "EI+Ei*")
  for i,y in enumerate((Eplot, ESplot, Pplot, EIplot, EIiplot, EItot)):
    axes[n,0].plot(y, label=labels[i])
  axes[n,0].legend()
  for i,y in enumerate((Eplot, ESplot, Pplot, EIplot, EIiplot, EItot)):
      axes[n,1].plot(y, label=labels[i])
  axes[n,1].legend()
  axes[n,1].set_ylim(0,E0) #zoom in tot E0
  axes[n,0].set_title("[I] = {}".format(str(Is[n])))
  axes[n,1].set_title("[I] = {}".format(str(Is[n])))
  
  df = pd.DataFrame(data={'E':Eplot,  "EI":EIplot, "EIi":EIiplot, "ES":ESplot, "P":Pplot})
  dataframes.append(df)
df = pd.concat(dataframes, keys=[I for I in Is], axis=1) # Combine all the dataframes

In [0]:
df

In [0]:
df.iloc[30]